# Calculo de las metricas en python

### Importacion de las librerias necesarias

In [4]:
import json
import re
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

### Lectura del documento

In [7]:
# Ruta del archivo con los datos obtenidos por el "Tracking system"
file_path = './RESULT.json'

In [17]:
# Abrir el archivo json y cargarlo en la variable data
with open(file_path) as f:
    data = json.load(f)

# Función para convertir el formato de fecha específico de .NET a datetime de Python
def parse_dotnet_datetime(dotnet_datetime):
    parts = dotnet_datetime.split('(')
    milliseconds = int(parts[1].split('+')[0])
    
    return milliseconds

# Función para convertir las fechas en la lista de diccionarios
def parse_json_with_dotnet_datetime(data):
    for item in data:
        if 'timestamp' in item:
            item['timestamp'] = parse_dotnet_datetime(item['timestamp'])
    return data

# Deserializar JSON
parsed_data = parse_json_with_dotnet_datetime(data)


# Se ordena la información del json en base el timestamp
sorted_data = sorted(data, key=lambda x: x['timestamp'])


print(parsed_data)  # Esto mostrará el diccionario con la fecha convertida a datetime de Python

[{'id': '64a55472a03465b47adb66460a950fed4ca78e427be8dbb122f71ee4fb3ab44f', 'idLevel': 0, 'idSession': '7e201f0c2cb3679a52cf76fe0f3fb74bc27f67283f49619f7256c4c2e3d1c7aa', 'idUser': '49562dc51f833b120a5fa281ca787e3eb4be7369402b1eb72eb7453e172e20b8', 'timestamp': 1713946301159, 'typeEvent': 'LoginEvent'}, {'id': '16db91c4c77fa4620d0fb36600464517b2d3661e3194f8334728c3a2a6dbc4ca', 'idLevel': 0, 'idSession': '7e201f0c2cb3679a52cf76fe0f3fb74bc27f67283f49619f7256c4c2e3d1c7aa', 'idUser': '49562dc51f833b120a5fa281ca787e3eb4be7369402b1eb72eb7453e172e20b8', 'timestamp': 1713946306139, 'typeEvent': 'StartGameEvent'}, {'id': 'f871613d05335a244192e81eb383de912059680d4f1223d5d47507a1132a4d9a', 'idLevel': 0, 'idSession': '7e201f0c2cb3679a52cf76fe0f3fb74bc27f67283f49619f7256c4c2e3d1c7aa', 'idUser': '49562dc51f833b120a5fa281ca787e3eb4be7369402b1eb72eb7453e172e20b8', 'timestamp': 1713946349290, 'typeEvent': 'LoseShieldEvent'}, {'id': 'c0f1e40e46d02a87a5bf518cc89d64ba90892866e72ea6235c845075ab6a2057', 'id

In [ ]:
# Longitud de cada sesion en milisegundos
gameSessionLengthMs = []
for currentEvent in sorted_data:
    if currentEvent['typeEvent'] == "LoginEvent":
        tsSessionStart = currentEvent['timestamp']
        r = re.search(r'\d+', tsSessionStart)
        numeric_tsSessionStart = int(r.group())
    if currentEvent['typeEvent'] == "LogoutEvent":
        tsSessionEnd = currentEvent['timestamp']
        r = re.search(r'\d+', tsSessionEnd)
        numeric_tsSessionEnd = int(r.group())
        gameSessionLengthMs.append(numeric_tsSessionEnd- numeric_tsSessionStart)

s = pd.Series(gameSessionLengthMs)
#print(s.describe())

count         1.0
mean     229048.0
std           NaN
min      229048.0
25%      229048.0
50%      229048.0
75%      229048.0
max      229048.0
dtype: float64


In [ ]:
s

0    229048
dtype: int64

In [ ]:
index = 0
num_events = len(sorted_data)

while index < num_events:
    currentEvent = sorted_data[index]
    consumeEvent = currentContext.parseEvent(currentEvent)
    if consumeEvent:
        index+=1

In [ ]:
#def parseEvent(event):
    